## Importing necessary modules

In [ ]:
import pandas as pd
import pymysql

## Reading the CSV Files 

In [ ]:
stores=pd.read_csv("Dataset-csv/Stores.csv")
products=pd.read_csv("Dataset-csv/Products.csv")
#Encoding used as names contain some non-ascii characters
customers=pd.read_csv("Dataset-csv/Customers.csv", encoding='ISO-8859-1')
sales=pd.read_csv("Dataset-csv/Sales.csv")
exchange=pd.read_csv("Dataset-csv/Exchange_Rates.csv")

## Data Cleaning for stores dataset 

In [ ]:
#print stores datset
stores

In [ ]:
#Checking the datatypes
stores.dtypes

In [ ]:
#Converting Object types to respective types

#Converting open date attribute to date data type
stores['Open Date']=pd.to_datetime(stores['Open Date'],format='%m/%d/%Y')

#Converting Country and state attribute to string datatype
stores['Country']=stores['Country'].astype(str)
stores['State']=stores['State'].astype(str)

In [ ]:
#Check if there are any null values in any column
stores.isnull().sum()

In [ ]:
#Get the record that has null value in Square Meter Attribute
null_records = stores[stores['Square Meters'].isnull()]
null_records

In [ ]:
#Fill 0 for the missing value
stores.fillna(0,inplace=True)

## Data Cleaning for products dataset 

In [ ]:
#printing product dataset
products

In [ ]:
#Checking the datatypes
products.dtypes

In [ ]:
#Removing the $ symbol
products['Unit Cost USD'] = products['Unit Cost USD'].replace('[\$,]', '', regex=True).astype(float)
products['Unit Price USD'] = products['Unit Price USD'].replace('[\$,]', '', regex=True).astype(float)

In [ ]:
#Check for null values
products.isnull().sum()

## Data Cleaning for customers dataset

In [ ]:
#Printing the customers dataset
customers

In [ ]:
#Checking the datatypes
customers.dtypes

In [ ]:
#Converting Object types to respective types

#Converting birthday attribute to date data type
customers['Birthday']=pd.to_datetime(customers['Birthday'],format='%m/%d/%Y')


In [ ]:
#Checking for null values
customers.isnull().sum()

In [ ]:
#Get the null records for state code
null_records = customers[customers['State Code'].isnull()]
null_records

In [ ]:
#Replace null values with not available
customers['State Code'].fillna('Not Available', inplace=True)

## Data Cleaning for sales dataset 

In [ ]:
#Printing the sales dataset
sales

In [ ]:
#Checking the datatypes
sales.dtypes

In [ ]:
#Converting the object types to date type
sales['Order Date']=pd.to_datetime(sales['Order Date'],format='%m/%d/%Y')
sales['Delivery Date']=pd.to_datetime(sales['Delivery Date'],format='%m/%d/%Y')

#Converting Currency code to str type
sales['Currency Code'] = sales['Currency Code'].astype(str)

In [ ]:
#Check for null values
sales.isnull().sum()

In [ ]:
#Fill the missing delivery dates with yet to be delivered
sales.fillna('Not Delivered', inplace=True)

## Data cleaning for exchage rates 

In [ ]:
#Printing the dataset
exchange

In [ ]:
#Checking the datatypes
exchange.dtypes

In [ ]:
#Conversion of datatypes
exchange['Date'] = pd.to_datetime(exchange['Date'], format='%m/%d/%Y')
exchange['Currency'] = exchange['Currency'].astype(str)

In [ ]:
#Checking for null values
exchange.isnull().sum()

## Merging all the datasets using inner join 

In [ ]:
#Merging customers and sales
globalElectronics=customers
globalElectronics=pd.merge(globalElectronics,sales,on='CustomerKey',how='inner')
globalElectronics

In [ ]:
#Merging all other datasets
globalElectronics=pd.merge(globalElectronics,exchange,left_on=['Order Date','Currency Code'],right_on=['Date','Currency'],how='inner')
globalElectronics=pd.merge(globalElectronics,products,on='ProductKey',how='inner')
globalElectronics=pd.merge(globalElectronics,stores,on=['StoreKey','Country','State'],how='inner')
globalElectronics

In [ ]:
#Check for null values in merged dataset
globalElectronics.isnull().sum()

## Converting column names to python convention 

In [ ]:
#For combined dataset
column=list(globalElectronics.columns)
cols=[]
for col in column:
    cols.append(col.replace(' ','_').lower())
    
globalElectronics.columns=cols

In [ ]:
#For stores dataset
column=list(stores.columns)
cols=[]
for col in column:
    cols.append(col.replace(' ','_').lower())
    
stores.columns=cols
cols

In [ ]:
#For poducts dataset
column=list(products.columns)
cols=[]
for col in column:
    cols.append(col.replace(' ','_').lower())
    
products.columns=cols
cols

In [ ]:
#For customers dataset
column=list(customers.columns)
cols=[]
for col in column:
    cols.append(col.replace(' ','_').lower())
    
customers.columns=cols
cols

In [ ]:
#For sales dataset
column=list(sales.columns)
cols=[]
for col in column:
    cols.append(col.replace(' ','_').lower())
    
sales.columns=cols
cols

In [ ]:
#For exchange rates dataset
column=list(exchange.columns)
cols=[]
for col in column:
    cols.append(col.replace(' ','_').lower())
    
exchange.columns=cols
cols

## Save merged dataset as a csv file

In [ ]:
globalElectronics.to_csv('globalElectronics.csv')

## Connecting to MySQL 

In [ ]:
conn = pymysql.connect(
    host="localhost",
    user="root",
    password="Sky1234",
    database="Global_Electronics",
    autocommit=True
)

print("Connected to the database!")
print(conn)
my_cursor = conn.cursor()

## Creating Tables and inserting records 

In [ ]:
# Table Creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS Overall(
                  ID INT AUTO_INCREMENT PRIMARY KEY,
                  Customerkey INT NOT NULL,
                  gender VARCHAR(255) NOT NULL,
                  name VARCHAR(255) NOT NULL,
                  city VARCHAR(255) NOT NULL,
                  state_code VARCHAR(255) NOT NULL,
                  state VARCHAR(255) NULL,
                  zip_code VARCHAR(255) NOT NULL,
                  country VARCHAR(255) NULL,
                  continent VARCHAR(255) NULL,
                  birthday DATE NOT NULL,
                  order_number INT NOT NULL,
                  line_item INT NOT NULL,
                  order_date DATE NOT NULL,
                  delivery_date VARCHAR(255) NOT NULL,
                  storekey INT NOT NULL,
                  productkey INT NOT NULL,
                  quantity INT NOT NULL,
                  currency_code VARCHAR(255) NOT NULL,
                  date DATE NOT NULL,
                  currency VARCHAR(255) NOT NULL,
                  exchange FLOAT NOT NULL,
                  product_name VARCHAR(255) NOT NULL,
                  brand VARCHAR(255) NOT NULL,
                  color VARCHAR(255) NOT NULL,
                  unit_cost_usd FLOAT NOT NULL,
                  unit_price_usd FLOAT NOT NULL,
                  subcategorykey INT NOT NULL,
                  subcategory VARCHAR(255) NOT NULL,
                  categorykey INT NOT NULL,
                  category VARCHAR(255) NOT NULL,
                  square_meters FLOAT NOT NULL,
                  open_date DATE NOT NULL
                  )''')
print("Table Created successfully")

In [ ]:
insert_query = '''INSERT INTO Overall(
                      Customerkey,
                      gender,
                      name,
                      city,
                      state_code,
                      state,
                      zip_code,
                      country,
                      continent,
                      birthday,
                      order_number,
                      line_item,
                      order_date,
                      delivery_date,
                      storekey,
                      productkey,
                      quantity,
                      currency_code,
                      date,
                      currency,
                      exchange,
                      product_name,
                      brand,
                      color,
                      unit_cost_usd,
                      unit_price_usd,
                      subcategorykey,
                      subcategory,
                      categorykey,
                      category,
                      square_meters,
                      open_date)
                  VALUES(%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s)'''

data = globalElectronics.values.tolist()

my_cursor.executemany(insert_query, data)

conn.commit()

print("Values inserted successfully")

In [ ]:
# Table Creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS Stores(
                  storekey INT PRIMARY KEY,
                  state VARCHAR(255) NULL,
                  country VARCHAR(255) NULL,
                  square_meters FLOAT NOT NULL,
                  open_date DATE NOT NULL
                        
                  )''')
print("Table Created successfully")

In [ ]:
insert_query = '''INSERT INTO Stores(
                      storekey,
                      state,
                      country,
                      square_meters,
                      open_date)
                  VALUES(%s, %s, %s, %s, %s)'''

data = stores.values.tolist()

my_cursor.executemany(insert_query, data)
conn.commit()

print("Values inserted successfully")

In [ ]:
# Table Creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS Products(
                  productkey INT PRIMARY KEY,
                  product_name VARCHAR(255) NOT NULL,
                  brand VARCHAR(255) NOT NULL,
                  color VARCHAR(255) NOT NULL,
                  unit_cost_usd FLOAT NOT NULL,
                  unit_price_usd FLOAT NOT NULL,
                  subcategorykey INT NOT NULL,
                  subcategory VARCHAR(255) NOT NULL,
                  categorykey INT NOT NULL,
                  category VARCHAR(255) NOT NULL
                        
                  )''')
print("Table Created successfully")

In [ ]:
insert_query = '''INSERT INTO Products(
                      productkey,
                      product_name,
                      brand,
                      color,
                      unit_cost_usd,
                      unit_price_usd,
                      subcategorykey,
                      subcategory,
                      categorykey,
                      category)
                  VALUES(%s, %s, %s, %s, %s,%s, %s, %s, %s, %s)'''

data = products.values.tolist()

my_cursor.executemany(insert_query, data)

conn.commit()

print("Values inserted successfully")

In [ ]:
# Table Creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS Customer(
                  Customerkey INT PRIMARY KEY,
                  gender VARCHAR(255) NOT NULL,
                  name VARCHAR(255) NOT NULL,
                  city VARCHAR(255) NOT NULL,
                  state_code VARCHAR(255) NOT NULL,
                  state VARCHAR(255) NULL,
                  zip_code VARCHAR(255) NOT NULL,
                  country VARCHAR(255) NULL,
                  continent VARCHAR(255) NULL,
                  birthday DATE NOT NULL
                        
                  )''')
print("Table Created successfully")

In [ ]:
insert_query = '''INSERT INTO Customer(
                      Customerkey,
                      gender,
                      name,
                      city,
                      state_code,
                      state,
                      zip_code,
                      country,
                      continent,
                      birthday)
                  VALUES(%s, %s, %s, %s, %s,%s, %s, %s, %s, %s)'''

data = customers.values.tolist()

my_cursor.executemany(insert_query, data)
conn.commit()

print("Values inserted successfully")

In [ ]:
# Table Creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS Sales(
                    ID INT AUTO_INCREMENT PRIMARY KEY,
                  order_number INT NOT NULL,
                  line_item INT NOT NULL,
                  order_date DATE NOT NULL,
                  delivery_date VARCHAR(255) NOT NULL,
                  Customerkey INT NOT NULL,
                  storekey INT NOT NULL,
                  productkey INT NOT NULL,
                  quantity INT NOT NULL,
                  currency_code VARCHAR(255) NOT NULL
                        
                  )''')
print("Table Created successfully")

In [ ]:
insert_query = '''INSERT INTO Sales(
                      order_number,
                      line_item,
                      order_date,
                      delivery_date,
                      Customerkey,
                      storekey,
                      productkey,
                      quantity,
                      currency_code)
                  VALUES(%s, %s, %s, %s, %s,%s, %s, %s, %s)'''

data = sales.values.tolist()

my_cursor.executemany(insert_query, data)
conn.commit()

print("Values inserted successfully")

In [ ]:
# Table Creation
my_cursor.execute('''CREATE TABLE IF NOT EXISTS Exchange(
                    date DATE NOT NULL,
                  currency VARCHAR(255) NOT NULL,
                  exchange FLOAT NOT NULL
                        
                  )''')
print("Table Created successfully")

In [ ]:
insert_query = '''INSERT INTO Exchange(
                      date,
                  currency,
                  exchange)
                  VALUES(%s, %s, %s)'''

data = exchange.values.tolist()

my_cursor.executemany(insert_query, data)
conn.commit()

print("Values inserted successfully")